In [40]:
%dependency /add com.microsoft.semantic-kernel:semantickernel-api:1.0.0-rc2
%dependency /add com.microsoft.semantic-kernel:semantickernel-bom:1.0.0-rc2
%dependency /add com.microsoft.semantic-kernel:semantickernel-parent:1.0.0-rc2
%dependency /add com.microsoft.semantic-kernel:semantickernel-aiservices-openai:1.0.0-rc2
%dependency /add org.slf4j:slf4j-jdk14:2.0.12

%dependency /resolve

Adding dependency com.microsoft.semantic-kernel:semantickernel-api:1.0.0-rc2
Adding dependency com.microsoft.semantic-kernel:semantickernel-bom:1.0.0-rc2
Adding dependency com.microsoft.semantic-kernel:semantickernel-parent:1.0.0-rc2
Adding dependency com.microsoft.semantic-kernel:semantickernel-aiservices-openai:1.0.0-rc2
Adding dependency org.slf4j:slf4j-jdk14:2.0.12
Solving dependencies
Found dependencies count: 42
Add to classpath: /Users/lokinfey/Library/Jupyter/kernels/rapaio-jupyter-kernel/ivy_cache/com.microsoft.semantic-kernel/semantickernel-api/jars/semantickernel-api-1.0.0-rc2.jar
Add to classpath: /Users/lokinfey/Library/Jupyter/kernels/rapaio-jupyter-kernel/ivy_cache/com.azure/azure-ai-openai/jars/azure-ai-openai-1.0.0-beta.6.jar
Add to classpath: /Users/lokinfey/Library/Jupyter/kernels/rapaio-jupyter-kernel/ivy_cache/com.azure/azure-core/jars/azure-core-1.45.1.jar
Add to classpath: /Users/lokinfey/Library/Jupyter/kernels/rapaio-jupyter-kernel/ivy_cache/com.azure/azure-jso

In [41]:
import java.nio.file.Path;

In [42]:
import com.azure.ai.openai.OpenAIAsyncClient;
import com.azure.ai.openai.OpenAIClientBuilder;
import com.azure.core.credential.AzureKeyCredential;
import com.azure.core.credential.KeyCredential;

In [43]:
import com.microsoft.semantickernel.Kernel;
import com.microsoft.semantickernel.Kernel.Builder;
import com.microsoft.semantickernel.exceptions.ConfigurationException;
import com.microsoft.semantickernel.orchestration.FunctionResult;
import com.microsoft.semantickernel.orchestration.PromptExecutionSettings;
import com.microsoft.semantickernel.plugin.KernelPluginFactory;
import com.microsoft.semantickernel.semanticfunctions.KernelFunction;
import com.microsoft.semantickernel.semanticfunctions.KernelFunctionArguments;
import com.microsoft.semantickernel.semanticfunctions.KernelFunctionFromPrompt;
import com.microsoft.semantickernel.services.textcompletion.TextGenerationService;

import com.microsoft.semantickernel.aiservices.openai.chatcompletion.OpenAIChatCompletion;
import com.microsoft.semantickernel.services.chatcompletion.ChatCompletionService;

In [44]:
String AZURE_KEY = "Your Azure OpenAI Service API Key";
String AZURE_ENDPOINT = "Your Azure OpenAI Service Endpoint";

In [45]:
OpenAIAsyncClient client = new OpenAIClientBuilder()
        .credential(new AzureKeyCredential(AZURE_KEY))
        .endpoint(AZURE_ENDPOINT)
        .buildAsyncClient();

In [46]:
ChatCompletionService chatGenerationService = OpenAIChatCompletion.builder()
                        .withOpenAIAsyncClient(client)
                        .withModelId("Your Azure OpenAI Service gpt-3.5-turbo-16k deployment name")
                        .build();

In [47]:

var codePlugin = KernelFunctionFromPrompt.builder()
                        .withTemplate("System: You are a java developer 。 User:{{$input}}")
                        .withDefaultExecutionSettings(
                            PromptExecutionSettings.builder()
                                .withMaxTokens(2000)
                                .withTemperature(0.7)
                                .build())
                        .build();

In [48]:
Kernel kernel = Kernel.builder()
    .withAIService(ChatCompletionService.class, chatGenerationService)
    .build();

In [49]:
var result = kernel
            .invokeAsync(codePlugin)
            .withArguments(
                KernelFunctionArguments.builder()
                    .withInput("Generate a bubble algorithm method with Java")
                    .build())
            .block();

In [50]:
result.getResult();

Sure, here's an example of a bubble sort algorithm implemented in Java:

```java
public class BubbleSort {
    public static void bubbleSort(int[] arr) {
        int n = arr.length;
        boolean swapped;
        for (int i = 0; i < n - 1; i++) {
            swapped = false;
            for (int j = 0; j < n - i - 1; j++) {
                if (arr[j] > arr[j + 1]) {
                    // Swap arr[j] and arr[j + 1]
                    int temp = arr[j];
                    arr[j] = arr[j + 1];
                    arr[j + 1] = temp;
                    swapped = true;
                }
            }
            // If no two elements were swapped in the inner loop, then the array is already sorted
            if (!swapped) {
                break;
            }
        }
    }

    public static void main(String[] args) {
        int[] arr = {64, 34, 25, 12, 22, 11, 90};

        System.out.println("Array before sorting:");
        for (int i : arr) {
            System.out.print(i + "

In [51]:
var translatePlugin =  KernelPluginFactory
                    .importPluginFromDirectory(
                        Path.of("../../../", "plugins"),
                        "TranslatePlugin",
                        null);

In [52]:
var result = kernel
            .invokeAsync(translatePlugin.get("MultiLanguage"))
            .withArguments(
                KernelFunctionArguments.builder()
                    .withVariable("input", "hello world")
                    .withVariable("language", "fr")
                    .build())
            .block();

In [53]:
result.getResult();

Bonjour tout le monde

In [54]:
import com.microsoft.semantickernel.semanticfunctions.annotations.DefineKernelFunction;
import com.microsoft.semantickernel.semanticfunctions.annotations.KernelFunctionParameter;

In [55]:
public static class CustomPlugin {

    @DefineKernelFunction(description = "Get news from the web", name = "NewsFunction")
    public static String askNews(
        @KernelFunctionParameter(description = "getlocation", name = "location") String location) {
        return "Get news from the " + location + ".";
    }

    @DefineKernelFunction(description = "Search Weather in a city", name = "WeatherFunction")
    public String askWeather(
        @KernelFunctionParameter(description = "get city", name = "city") String city) {
        return city + "’s weather is 30 celsius degree , and very hot.";
    }

    @DefineKernelFunction(description = "Search docs", name = "DocsFunction")
    public String askDocs(
        @KernelFunctionParameter(description = "get docs", name = "docs") String docs) {
        return "ask docs :" + docs;
    }
}

In [56]:
Builder kernelBuilder = Kernel.builder()
            .withAIService(ChatCompletionService.class, chatGenerationService);

In [57]:

var customPlugin =KernelPluginFactory.createFromObject(new CustomPlugin(),"CustomPlugin")

In [58]:
kernelBuilder.withPlugin(customPlugin);

com.microsoft.semantickernel.Kernel$Builder@431dbfc2

In [59]:
Kernel kernel = kernelBuilder.build();

In [60]:

var result = kernel.invokeAsync(kernel.getFunction("CustomPlugin", "NewsFunction"))
                        .withArguments(
                            KernelFunctionArguments.builder()
                                .withVariable("location", "guangzhou")
                                .build())
                        .block();

In [61]:
result.getResult();

Get news from the guangzhou.